In [2]:
import os
import cv2
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from pathlib import Path

import psutil
import GPUtil
import platform

Establecemos una función que permita conocer las características del documento.

In [4]:
def know_pc_system_info():
    print("Modelo del equipo Dell G5 5590")
    print(f"Información del PC: {platform.uname()}")
    print("------------------------------")
    # Para obtener información de la CPU
    print("Núcleos físicos CPU:", psutil.cpu_count(logical=False))
    print("Todos los núcleos CPU:", psutil.cpu_count(logical=True))
    print("Frecuencia de la CPU:", psutil.cpu_freq().current)

    # Para obtener información de la RAM
    memoria = psutil.virtual_memory()
    print("Total RAM:", memoria.total)
    print("Disponible RAM:", memoria.available)

    # Para obtener información de la GPU (solo para GPUs NVIDIA)
    gpus = GPUtil.getGPUs()
    if gpus:
        gpu = gpus[0]
        print("GPU:", gpu.name)
        print("Total memoria de la GPU:", gpu.memoryTotal)

In [5]:
# Obtener el directorio actual (donde se encuentra test-model.py)
current_directory = Path.cwd()

# Construir la ruta hacia first-model.h5
models_path = current_directory.parent / 'models'
test_images_path = current_directory.parent / 'test'

In [6]:
# Función para cargar y preprocesar una imagen
def convert_image(path,size_img):
    print(path)
    # Lee la imagen desde la ruta especificada
    img = cv2.imread(path)
    # Convierte la imagen de BGR (formato por defecto de OpenCV) a RGB
    img_color = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Redimensiona la imagen a un tamaño específico (150x150 píxeles en este caso)
    resized_img = cv2.resize(img_color, (size_img, size_img))
    # Normaliza los valores de píxeles de la imagen de 0 a 1
    normalized_img = resized_img / 255.0
    # Reorganiza la imagen para que tenga la forma adecuada (1, 150, 150, 3)
    good_image = np.reshape(normalized_img, (1, size_img, size_img, 3))
    
    return good_image

# Cargamos los modelos entrenados previamente

## 🌌 Primer modelo cargado

In [29]:
# Cargamos nuestro primer modelo construido de acuerdo con investigaciones relacionadas con señales de tránsito.
first_own_model = load_model(f"{models_path}/first-model.h5")
# Imprimimos el resumen del modelo entrenado.
first_own_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 146, 146, 8)       608       
                                                                 
 conv2d_61 (Conv2D)          (None, 142, 142, 8)       1608      
                                                                 
 max_pooling2d_30 (MaxPooli  (None, 71, 71, 8)         0         
 ng2D)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 71, 71, 8)         0         
                                                                 
 conv2d_62 (Conv2D)          (None, 67, 67, 16)        3216      
                                                                 
 conv2d_63 (Conv2D)          (None, 63, 63, 16)        6416      
                                                     

In [16]:
# Lee las imágenes y las preprocesa utilizando la función 'convert_image'
image_Test_Speed = convert_image(f"{test_images_path}/speed-1.jpg",150)
image_Test_Stop = convert_image(f"{test_images_path}/stop-1.jpg",150)
image_Test_Traffic_Light = convert_image(f"{test_images_path}/traffic-light-1.jpg",150)

e:\Traffic-Sign-Classifier-MODELS\test/speed-1.jpg
e:\Traffic-Sign-Classifier-MODELS\test/stop-1.jpg
e:\Traffic-Sign-Classifier-MODELS\test/traffic-light-1.jpg


In [18]:
# Predicción para la imagen de velocidad
print("SPEED == 0")
start = time.time()
y_pred_speed = first_own_model.predict(image_Test_Speed)
finish = time.time()
# Imprime la clase predicha (0: Speed)
print("Predicción: ", y_pred_speed.argmax()) 
print(f"Speed se demoró: {finish - start} segundos.")
print("--------------------------------------------------------------------")

# Predicción para la imagen de stop
print("STOP == 1")
start = time.time()
y_pred_stop = first_own_model.predict(image_Test_Stop)
finish = time.time()
# Imprime la clase predicha (1: Stop)
print("Predicción: ", y_pred_stop.argmax())  
print(f"Stop se demoró: {finish - start} segundos.")
print("--------------------------------------------------------------------")

# Predicción para la imagen de semáforo
print("TRAFFIC LIGHT == 2")
start = time.time()
y_pred_trafficlight = first_own_model.predict(image_Test_Traffic_Light)
finish = time.time()
# Imprime la clase predicha (2: Traffic-Light)
print("Predicción: ", y_pred_trafficlight.argmax())  
print(f"Traffic Light se demoró: {finish - start} segundos.")
print("--------------------------------------------------------------------")

know_pc_system_info()

SPEED == 0
1/1 [==============================] - 0s 23ms/step
Predicción:  0
Speed se demoró: 0.07200098037719727 segundos.
--------------------------------------------------------------------
STOP == 1
1/1 [==============================] - 0s 32ms/step
Predicción:  1
Stop se demoró: 0.08001208305358887 segundos.
--------------------------------------------------------------------
TRAFFIC LIGHT == 2
1/1 [==============================] - 0s 22ms/step
Predicción:  2
Traffic Light se demoró: 0.05899977684020996 segundos.
--------------------------------------------------------------------
Modelo del equipo Dell G5 5590
Información del PC: uname_result(system='Windows', node='DESKTOP-KKKLEC7', release='10', version='10.0.19045', machine='AMD64')
------------------------------
Núcleos físicos CPU: 6
Todos los núcleos CPU: 12
Frecuencia de la CPU: 2592.0
Total RAM: 16983531520
Disponible RAM: 4081831936
GPU: NVIDIA GeForce GTX 1660 Ti
Total memoria de la GPU: 6144.0


## 🚀 Segundo modelo cargado - Basado en arquitectura de internet

In [30]:
# Cargamos nuestro segundo modelo construido de acuerdo a una arquitectura de internet.
internet_architecture_model = load_model(f"{models_path}/VGG16_v1.h5")
# Imprimimos el resumen del modelo entrenado.
internet_architecture_model.summary()

Model: "sequential_52"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_646 (Conv2D)         (None, 224, 224, 8)       224       
                                                                 
 batch_normalization_262 (B  (None, 224, 224, 8)       32        
 atchNormalization)                                              
                                                                 
 conv2d_647 (Conv2D)         (None, 224, 224, 8)       584       
                                                                 
 batch_normalization_263 (B  (None, 224, 224, 8)       32        
 atchNormalization)                                              
                                                                 
 max_pooling2d_250 (MaxPool  (None, 112, 112, 8)       0         
 ing2D)                                                          
                                                     

In [20]:
# Lee las imágenes y las preprocesa utilizando la función 'convert_image'
image_Test_Speed = convert_image(f"{test_images_path}/speed-1.jpg",224)
image_Test_Stop = convert_image(f"{test_images_path}/stop-1.jpg",224)
image_Test_Traffic_Light = convert_image(f"{test_images_path}/traffic-light-1.jpg",224)

e:\Traffic-Sign-Classifier-MODELS\test/speed-1.jpg
e:\Traffic-Sign-Classifier-MODELS\test/stop-1.jpg
e:\Traffic-Sign-Classifier-MODELS\test/traffic-light-1.jpg


In [22]:
# Predicción para la imagen de velocidad
print("SPEED == 0")
start = time.time()
y_pred_speed = internet_architecture_model.predict(image_Test_Speed)
finish = time.time()
# Imprime la clase predicha (0: Speed)
print("Predicción: ", y_pred_speed.argmax()) 
print(f"Speed se demoró: {finish - start} segundos.")
print("--------------------------------------------------------------------")

# Predicción para la imagen de stop
print("STOP == 1")
start = time.time()
y_pred_stop = internet_architecture_model.predict(image_Test_Stop)
finish = time.time()
# Imprime la clase predicha (1: Stop)
print("Predicción: ", y_pred_stop.argmax())  
print(f"Stop se demoró: {finish - start} segundos.")
print("--------------------------------------------------------------------")

# Predicción para la imagen de semáforo
print("TRAFFIC LIGHT == 2")
start = time.time()
y_pred_trafficlight = internet_architecture_model.predict(image_Test_Traffic_Light)
finish = time.time()
# Imprime la clase predicha (2: Traffic-Light)
print("Predicción: ", y_pred_trafficlight.argmax())  
print(f"Traffic Light se demoró: {finish - start} segundos.")
print("--------------------------------------------------------------------")

know_pc_system_info()

SPEED == 0
1/1 [==============================] - 0s 28ms/step
Predicción:  2
Speed se demoró: 0.07999825477600098 segundos.
--------------------------------------------------------------------
STOP == 1
1/1 [==============================] - 0s 27ms/step
Predicción:  1
Stop se demoró: 0.0709993839263916 segundos.
--------------------------------------------------------------------
TRAFFIC LIGHT == 2
1/1 [==============================] - 0s 30ms/step
Predicción:  2
Traffic Light se demoró: 0.07360982894897461 segundos.
--------------------------------------------------------------------
Modelo del equipo Dell G5 5590
Información del PC: uname_result(system='Windows', node='DESKTOP-KKKLEC7', release='10', version='10.0.19045', machine='AMD64')
------------------------------
Núcleos físicos CPU: 6
Todos los núcleos CPU: 12
Frecuencia de la CPU: 2592.0
Total RAM: 16983531520
Disponible RAM: 4046069760
GPU: NVIDIA GeForce GTX 1660 Ti
Total memoria de la GPU: 6144.0


## 👻 Tercer modelo - Trasnfer learning

In [31]:
# Cargamos nuestro tercer modelo construido por medio de Transfer learning del modelo Inception-v3.
transfer_learning_model = load_model(f"{models_path}/ModelV3_InceptionV3.h5")
# Imprimimos el resumen del modelo entrenado.
transfer_learning_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0]

In [25]:
# Lee las imágenes y las preprocesa utilizando la función 'convert_image'
image_Test_Speed = convert_image(f"{test_images_path}/speed-1.jpg",299)
image_Test_Stop = convert_image(f"{test_images_path}/stop-1.jpg",299)
image_Test_Traffic_Light = convert_image(f"{test_images_path}/traffic-light-1.jpg",299)

e:\Traffic-Sign-Classifier-MODELS\test/speed-1.jpg
e:\Traffic-Sign-Classifier-MODELS\test/stop-1.jpg
e:\Traffic-Sign-Classifier-MODELS\test/traffic-light-1.jpg


In [28]:
# Predicción para la imagen de velocidad
print("SPEED == 0")
start = time.time()
y_pred_speed =transfer_learning_model.predict(image_Test_Speed)
finish = time.time()
# Imprime la clase predicha (0: Speed)
print("Predicción: ", y_pred_speed.argmax()) 
print(f"Speed se demoró: {finish - start} segundos.")
print("--------------------------------------------------------------------")

# Predicción para la imagen de stop
print("STOP == 1")
start = time.time()
y_pred_stop =transfer_learning_model.predict(image_Test_Stop)
finish = time.time()
# Imprime la clase predicha (1: Stop)
print("Predicción: ", y_pred_stop.argmax())  
print(f"Stop se demoró: {finish - start} segundos.")
print("--------------------------------------------------------------------")

# Predicción para la imagen de semáforo
print("TRAFFIC LIGHT == 2")
start = time.time()
y_pred_trafficlight =transfer_learning_model.predict(image_Test_Traffic_Light)
finish = time.time()
# Imprime la clase predicha (2: Traffic-Light)
print("Predicción: ", y_pred_trafficlight.argmax())  
print(f"Traffic Light se demoró: {finish - start} segundos.")
print("--------------------------------------------------------------------")

know_pc_system_info()

SPEED == 0
1/1 [==============================] - 0s 81ms/step
Predicción:  0
Speed se demoró: 0.14199399948120117 segundos.
--------------------------------------------------------------------
STOP == 1
1/1 [==============================] - 0s 91ms/step
Predicción:  1
Stop se demoró: 0.13723540306091309 segundos.
--------------------------------------------------------------------
TRAFFIC LIGHT == 2
1/1 [==============================] - 0s 93ms/step
Predicción:  2
Traffic Light se demoró: 0.13801026344299316 segundos.
--------------------------------------------------------------------
Modelo del equipo Dell G5 5590
Información del PC: uname_result(system='Windows', node='DESKTOP-KKKLEC7', release='10', version='10.0.19045', machine='AMD64')
------------------------------
Núcleos físicos CPU: 6
Todos los núcleos CPU: 12
Frecuencia de la CPU: 2592.0
Total RAM: 16983531520
Disponible RAM: 3828355072
GPU: NVIDIA GeForce GTX 1660 Ti
Total memoria de la GPU: 6144.0
